In [ ]:
%load_ext autoreload
%autoreload 2

import seaborn as sns
sns.set_context('talk')
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%config InlineBackend.figure_format = 'retina'
from datetime import timedelta
import datetime as dt
import scipy
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
from matplotlib.dates import MonthLocator, WeekdayLocator, DateFormatter
import pickle

from get_data.get_euromomo_data import get_euromomo_data
from get_data.get_data import get_ecdc_data,\
                        get_data_UK,\
                        get_data_Spain, \
                        get_data_Austria, \
                        get_data_Malta, \
                        get_data_Italy, \
                        get_data_Belgium,\
                        get_data_France, \
                        get_data_Portugal, \
                        get_data_Berlin,\
                        get_data_Norway,\
                        get_data_Netherlands,\
                        get_data_Denmark,\
                        get_data_Switzerland,\
                        get_data_Ireland,\
                        get_data_Greece,\
                        get_data_Luxemburg,\
                        get_data_Estonia,\
                        get_data_Sweden,\
                        get_data_Hesse

## Get ECDC data and group all Europe

In [ ]:
ecdc_data=get_ecdc_data(continent='Europe')
ecdc_europe=ecdc_data.groupby(by='date',as_index=False).sum()
ecdc_europe.deaths.clip(0,inplace=True)
ecdc_europe['state']='Europe'
ecdc_europe['datasource']='ecdc.europa.eu'




## Print list of countries in ECDC Europe

In [ ]:

from pprint import pprint
countries=(str(ecdc_data.state.unique())).strip('[]')
print('Countries in this data set:\n', countries)


# Get data for individual Euromomo countries

In [ ]:
countries=['Austria', 'Belgium', 'Denmark', 'Estonia', 'Finland', 'France','Greece', 'Hungary','Ireland', 
           'Italy', 'Luxembourg', 'Malta', 'Netherlands', 'Norway', 'Portugal', 'Spain', 'Sweden', 
           'Switzerland', 'UK','Germany (Berlin)','Germany (Hesse)']





FI=ecdc_data[ecdc_data.state=='Finland']

HU=ecdc_data[ecdc_data.state=='Hungary']

ES=get_data_Spain()
AT=get_data_Austria()
MT=get_data_Malta()
IT=get_data_Italy()
BE=get_data_Belgium()
FR=get_data_France()
PT=get_data_Portugal()
Berlin=get_data_Berlin()
NO=get_data_Norway()
NL=get_data_Netherlands()
DK=get_data_Denmark()
CH=get_data_Switzerland()
IE=get_data_Ireland()
GR=get_data_Greece()
LU=get_data_Luxemburg()
EE=get_data_Estonia()
SE=get_data_Sweden()
Hesse=get_data_Hesse()
UK=get_data_UK()

df=pd.concat([FI,HU,ES,AT,MT,IT,BE,FR,PT,Berlin,
              NO,NL,DK,CH,IE,GR,LU,EE,SE,Hesse,UK])


if len(countries)==len(df.state.unique()):
    print('All Euromomo countries done')
else: print('Something is wrong')

# Convert all timestamps to same time zone
df['date']=pd.to_datetime(df.date,utc=True)
# Get list of countries and datasource



In [ ]:
# Inspect dataframe 
df

In [ ]:
datasources=df.drop_duplicates('state',ignore_index=True)
datasources=datasources[['state','datasource']]
datasources=datasources.sort_values(by='state').reset_index()
# Get only relevant columns

## Output the list of data sources

fig, ax = plt.subplots(figsize=(12, 5)) # set size frame
ax.xaxis.set_visible(False)  # hide the x axis
ax.yaxis.set_visible(False)  # hide the y axis
ax.set_frame_on(False) 

fig=pd.plotting.table(ax,datasources[['state','datasource']],loc='upper right')  # where df is your data frame
fig.set_fontsize(30)

plt.savefig('plots/Euromomo_data_sources.png',dpi=200)



In [ ]:

# Now group by date
df=df[['date','positive','deaths','total']].groupby(by='date',as_index=False).sum()

In [ ]:
df


# Get EU/EEA & UK test and other data

In [ ]:
# Get EU/EEA & UK Tests

from get_data.get_data_ecdc import get_data_ecdc_tests

EUtestdf=get_data_ecdc_tests()

# Get other data only for only EU/EEA & UK 
ecdc_EUtest=ecdc_data[ecdc_data.state.isin(list(EUtestdf.country.unique()))]

#from pprint import pprint
countries=(str(ecdc_EUtest.state.unique())).strip('[]')
print('EU cases & deaths: Countries in this data set:\n', countries)

ecdc_EUtest=ecdc_EUtest.groupby(by='date',as_index=False).sum()
ecdc_EUtest.deaths.clip(0,inplace=True)
ecdc_EUtest['state']='EU/EEA & UK'
ecdc_EUtest['datasource']='ecdc.europa.eu'
ecdc_EUtest['positive_avg']=ecdc_EUtest.positive.rolling(window=7,center=True).mean()

print('EU tests: Countries in this data set:\n', (str(EUtestdf.country.unique())).strip('[]'))

#EUtest['date']=pd.to_datetime(EUtest.date)
EUtest=EUtestdf.groupby('year_week',as_index=False).sum()

#(EUtest['year_week'].apply(str)).strip()

EUtest['date']=pd.to_datetime(EUtestdf.year_week.add('-0'), format='%Y-W%W-%w')-dt.timedelta(5)

from dataframe_utilities import ECDC_week2day

EUtestdaily=ECDC_week2day(EUtest,'tests_done','dailytests')
EUtestdaily['dailytestsavg']=EUtestdaily.dailytests.rolling(window=7,center=True).mean()

In [ ]:
EUtest

In [ ]:
EUtestdaily

## Estimate false positives


In [ ]:
EUtestdaily=(pd.merge(EUtestdaily,ecdc_EUtest,on='date',how='outer')).reset_index(drop=True)

In [ ]:
EUtestdaily.sort_values(by='date',inplace=True)

In [ ]:
EUtestdaily

In [ ]:
EU_cases_deaths_tests_fpr=EUtestdaily.rename(columns={'dailytests':'total'})
#EU_cases_deaths_tests_fpr.to_pickle("./EU_EEA_UK_df.pkl")

In [ ]:
from rogan_gladen.mk_hpd import mk_hpd
from rogan_gladen.mk_true_positive import mk_true_positive

df_for_fpr=pd.DataFrame({'date':EUtestdaily.date,'positive':EUtestdaily.positive_avg,
                        'total':EUtestdaily.dailytestsavg})
df_for_fpr.dropna(inplace=True)
df_for_fpr.drop(df_for_fpr[df_for_fpr.total<df_for_fpr.positive].index,inplace=True)
                        


In [ ]:
df_for_fpr

In [ ]:
obs_positive_trace,data=mk_true_positive(df_for_fpr)

In [ ]:
EU_true=mk_hpd(obs_positive_trace,data)

In [ ]:
EU_true

In [ ]:

#pickle.dump(EU_true, open( "EUtrue.p", "wb" ) )


In [ ]:
EUtestdailyfpr=EUtestdaily.merge(EU_true,on='date',how='outer')

In [ ]:
EUtestdaily.rename(columns={'dailytests':'total'},inplace=True)

In [ ]:
from get_plots import plot_fpr

plot_fpr(EUtestdaily,EUtestdailyfpr,country='EU/EEA & UK',
           datasource='Data source: www.ecdc.europa.eu',mynotes=True) #.plot(x='date',y=['positive','falsepositive','truepositive'])

In [ ]:

from get_plots import mirrorplot,mirrorplot_withtest
mirrorplot(ecdc_EUtest,country='EU/EEA & UK',
           datasource='Data source: www.ecdc.europa.eu',mirror=False,mynotes=True)
#mirrorplot(euromomo_all,state='Euromomo countries',datasource='(From respective national data sources)',mirror=False)



In [ ]:
mirrorplot_withtest(ecdc_EUtest,EUtestdaily,country='EU/EEA & UK',
           datasource='Data source: www.ecdc.europa.eu',mirror=False,mynotes=True)

In [ ]:
# Get total of Covid deaths
covid=df.groupby('date',as_index=False)[('deaths','positive')].agg('sum')
covid['date']=covid.date.dt.tz_localize(None)


fig, ax = plt.subplots(figsize=(15, 8))
plt.title('Official Covid numbers in Euromomo countries')
covid.plot(x='date',y='positive',ax=ax,color='blue',label='cases')#
covid.plot(x='date',y='deaths',ax=ax,color='red',label='deaths')#
plt.legend()
plt.ylabel('daily numbers')

fig.set_facecolor('w')

sns.despine;
plt.savefig('plots/OfficialCovid_CasesDeaths.png',dpi=250)
plt.show()


# Get total of Covid deaths
#covid=df.groupby('date',as_index=False)[('deaths','positive')].agg('sum')
#covid['date']=covid.date.dt.tz_localize(None)


fig, ax = plt.subplots(figsize=(15, 8))
plt.title('Official Covid deaths in Euromomo countries')

covid.plot(x='date',y='deaths',ax=ax,color='red',legend=False)#
#plt.legend()
plt.ylabel('daily deaths')

fig.set_facecolor('w')

sns.despine;
plt.savefig('plots/OfficialCovidDeaths.png',dpi=250)
plt.show()




In [ ]:
#weekly deaths
covidweekly= covid.resample('W', on='date').sum()
covidweekly.reset_index(level=0, inplace=True)

# Get Euromomo data

In [ ]:
data=get_euromomo_data()
data.tail()

#  Plot Euromomo deaths & baseline graph

In [ ]:
from scipy.signal import find_peaks, peak_prominences
peaks, _= find_peaks(data.deaths.values,prominence=(10000,None))
troughs, _= find_peaks(-data.deaths.values,prominence=(None,5000))
troughs=np.append(troughs,[data.index[-1]])
prominences = peak_prominences(data.deaths.values, peaks)[0]

contour_heights = data.deaths.iloc[peaks] - prominences

print(peaks)

In [ ]:
baseline_peaks, _= find_peaks(data.baseline.values,prominence=1000)
baseline_troughs, _= find_peaks(-data.baseline.values,prominence=1000)

print(baseline_peaks,baseline_troughs)


baseline_prominences = peak_prominences(data.baseline.values, baseline_peaks)[0]
print(baseline_prominences)

baseline_contour_heights = data.baseline.iloc[peaks] - baseline_prominences

In [ ]:
baseline_peaks

In [ ]:

fig, ax = plt.subplots(figsize=(15, 8))
data.plot(x="date", y="deaths",color='blue',ax=ax)
data.loc[troughs].plot.scatter(x='date',y='deaths',marker='v',color='green',s=150,ax=ax)
data.loc[peaks].plot.scatter(x='date',y='deaths',marker='^',color='red',s=150,ax=ax)
data.plot(x='date',y='baseline',color='gray',ax=ax)

data.loc[baseline_troughs].plot.scatter(x='date',y='baseline',marker='+',color='green',s=250,ax=ax)
data.loc[baseline_peaks].plot.scatter(x='date',y='baseline',marker='+',color='red',s=250,ax=ax)

ax.fill_between(data.date,data.baseline_upper,data.baseline_lower, alpha=0.35,color='gray')

sns.despine;
plt.show()

In [ ]:
#Find the cumulative sum of deaths in each peak

totaldeathsarray=[]

for i in np.arange(len(peaks)-1):
    totaldeaths=data.deaths.iloc[troughs[i]:troughs[i+1]].sum()
    totaldeathsarray.append(totaldeaths)

In [ ]:
#Find the cumulative sum of deaths between troughs in the baseline


baseline_totaldeathsarray=[]


for i in np.arange(len(baseline_peaks)-1):
    baseline_totaldeaths=data.deaths.iloc[baseline_troughs[i]:baseline_troughs[i+1]].sum()
    baseline_totaldeathsarray.append(baseline_totaldeaths)
    
    

In [ ]:
def weekdiff(start, end):
    days = pd.to_datetime(end) - pd.to_datetime(start)
    week = int(pd.Timedelta(days).days / 7)
    remainder = pd.Timedelta(days).days % 7
    #print(week)
    return week

trough_to_trough_interval=[]
intervaldates=[]

for i in np.arange(1,len(troughs)):

    trough_to_trough_interval.append(weekdiff(data.date.iloc[troughs[i-1]],data.date.iloc[troughs[i]]))
    intervaldates.append((data.date.iloc[troughs[i-1]].strftime("%d %b %Y") ,data.date.iloc[troughs[i]].strftime("%d %b %Y") ))

In [ ]:
baseline_trough_to_trough_interval=[]
baseline_intervaldates=[]

for i in np.arange(1,len(baseline_troughs)):

    baseline_trough_to_trough_interval.append(weekdiff(data.date.iloc[baseline_troughs[i-1]],
                                                       data.date.iloc[baseline_troughs[i]]))
    baseline_intervaldates.append((data.date.iloc[baseline_troughs[i-1]].strftime("%d %b %Y") ,
                                   data.date.iloc[baseline_troughs[i]].strftime("%d %b %Y") ))

In [ ]:
# Find the start and end points of the pandemic spike
pandemic=data[(data.date>pd.Timestamp('2020-02-26')) & (data.date<pd.Timestamp('2020-06-08'))]





data['noncovidbaseline']=data.deaths
data.loc[pandemic.index.values,'noncovidbaseline']=None

data['noncovidbaseline']=data['noncovidbaseline'].interpolate(method='akima')



In [ ]:
from dataframe_utilities import insert_row
last_index=pandemic.index[-1]
first_index=pandemic.index[0]

pandemic=insert_row(first_index-1,pandemic,data.iloc[first_index-1])
pandemic=insert_row(last_index+1,pandemic,data.iloc[last_index+1])
pandemic.sort_index(inplace=True)

In [ ]:
yearlyintervals=[]
for i in np.arange(4):
    yearlyintervals.append(data.date.iloc[-1-i*52])
    
    
for yearlyinterval in yearlyintervals: print(yearlyinterval)
    
year_to_year_interval=[]
for i in np.arange(1,len(yearlyintervals)):

    year_to_year_interval.append(weekdiff(yearlyintervals[i],yearlyintervals[i-1]))
    
yearlyindices=data[data.date.isin(yearlyintervals)].index
    
#Find the cumulative sum of deaths for every year starting from latest date in series, going backwards


yearly_totaldeathsarray=[]


for i in np.arange(len(yearlyintervals)-1):
    yearly_totaldeaths=data.deaths.iloc[yearlyindices[i]:yearlyindices[i+1]].sum()
    yearly_totaldeathsarray.append(yearly_totaldeaths)
    
    

In [ ]:
yearly_totaldeathsarray

In [ ]:
data.date[yearlyindices[0]]

In [ ]:

fig, ax = plt.subplots(figsize=(15, 8))

data.plot(x="date", y="deaths",color='blue',ax=ax,legend=True,label='Deaths')


wrtbaseline=False
wrtpeaks=False
wrtdate=True


if wrtpeaks:
    
    plt.title('All-age deaths in EUROMOMO countries')

    for i in np.arange(len(troughs)):
        plt.axvline(data.date.iloc[troughs[i]],ymax=0.9,color='gray',alpha=0.5,linestyle=':')
    for i in np.arange(len(troughs)-1):
        ax.annotate('{} - {}'.format(intervaldates[i][0],intervaldates[i][1]), xy=(data.date.iloc[peaks[i]], data.deaths.iloc[troughs[i]]  ),
                    xytext=(data.date.iloc[troughs[i]+5], 1.05*data.deaths.iloc[peaks[i]]+6000),size=12)
        ax.annotate('{:,} weeks'.format(trough_to_trough_interval[i]), xy=(data.date.iloc[peaks[i]], data.deaths.iloc[troughs[i]]  ),
                    xytext=(data.date.iloc[troughs[i]+5], 1.05*data.deaths.iloc[peaks[i]]+3000))
        ax.annotate('{:,} deaths'.format(int(totaldeathsarray[i])), xy=(data.date.iloc[peaks[i]], data.deaths.iloc[troughs[i]]  ),
                    xytext=(data.date.iloc[troughs[i]+5], 1.05*data.deaths.iloc[peaks[i]]))
        
elif wrtbaseline:
    
    plt.title('All-age deaths in EUROMOMO countries \n with total deaths between troughs in baseline')
    
    for i in np.arange(len(baseline_troughs)):
        plt.axvline(data.date.iloc[baseline_troughs[i]],ymax=0.9,color='gray',alpha=0.5,linestyle=':')
    for i in np.arange(len(baseline_troughs)-1):
        ax.annotate('{} - {}'.format(baseline_intervaldates[i][0],baseline_intervaldates[i][1]),
                    xy=(data.date.iloc[baseline_peaks[i]], data.deaths.iloc[baseline_troughs[i]]  ),
                    xytext=(data.date.iloc[baseline_troughs[i]+5], 
                            1.1*data.deaths.iloc[baseline_peaks[i]]+6000),size=12)
        ax.annotate('{:,} weeks'.format(baseline_trough_to_trough_interval[i]),
                    xy=(data.date.iloc[baseline_peaks[i]], data.deaths.iloc[baseline_troughs[i]]  ),
                    xytext=(data.date.iloc[baseline_troughs[i]+5], 1.1*data.deaths.iloc[baseline_peaks[i]]+3000))
        ax.annotate('{:,} deaths'.format(int(baseline_totaldeathsarray[i])), 
                    xy=(data.date.iloc[baseline_peaks[i]], data.deaths.iloc[baseline_troughs[i]]  ),
                    xytext=(data.date.iloc[baseline_troughs[i]+5], 1.1*data.deaths.iloc[baseline_peaks[i]]))
        
elif wrtdate:
    
    plt.title('All-age deaths in EUROMOMO countries \n with total deaths for 52-week period')
    
    for i in np.arange(1,len(yearlyindices)):
        plt.axvline(data.date.iloc[yearlyindices[i-1]],ymax=0.9,color='gray',alpha=0.5,linestyle=':')
    for i in np.arange(1,len(yearlyindices)):
        ax.annotate('{} - {}'.format(data.date.iloc[yearlyindices[i-1]].strftime("%d %b %Y"),data.date.iloc[yearlyindices[i]].strftime("%d %b %Y")),
                    xy=(data.date.iloc[yearlyindices[i-1]], data.deaths.iloc[yearlyindices[i-1]] ),
                    xytext=(data.date.iloc[yearlyindices[i-1]+5], 
                            1.3*data.deaths.iloc[yearlyindices[i]]+6000),size=12)
#         ax.annotate('{:,} weeks'.format(baseline_trough_to_trough_interval[i]),
#                     xy=(data.date.iloc[baseline_peaks[i]], data.deaths.iloc[baseline_troughs[i]]  ),
#                     xytext=(data.date.iloc[baseline_troughs[i]+5], 1.1*data.deaths.iloc[baseline_peaks[i]]+3000))
        ax.annotate('{:,} deaths'.format(int(yearly_totaldeathsarray[i-1])), 
                    xy=(data.date.iloc[yearlyindices[i-1]], data.deaths.iloc[yearlyindices[i-1]] ),
                    xytext=(data.date.iloc[yearlyindices[i-1]+5], 1.6*data.deaths.iloc[yearlyindices[i]]))
            
    
#sns.set(font_scale=5) 
#sns.set_style("ticks", {"xtick.major.size": 8, "ytick.major.size": 8})




ax.xaxis.set_major_locator(mdates.YearLocator())

ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))

ax.xaxis.set_minor_locator(mdates.MonthLocator())


#plt.vlines(x=data.date.iloc[peaks], ymin=contour_heights, ymax=data.deaths.iloc[peaks])

#ax.xaxis.set_minor_locator(mdates.WeekdayLocator())
#set major ticks format
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
ax.get_xaxis().set_tick_params(which='major', pad=5)

data.plot(x='date',y='baseline',color='orange',ax=ax,label='Euromomo baseline',alpha=0.5)

ax.fill_between(data.date,data.baseline_upper,data.baseline_lower, alpha=0.35,color='orange')


#x.plot(x="date", y="deaths",color='green',ax=ax,legend=False)


ax.legend(fancybox=True, framealpha=1, shadow=False, borderpad=1,
          loc='upper left',fontsize='small',facecolor='white')

fig.set_facecolor('w')

#plt.show()
plt.tight_layout();
sns.despine;


plt.savefig('plots/Euromomo_total_deaths.jpg',dpi=250)
 

In [ ]:
totalcoviddeaths=(data.deaths-data.noncovidbaseline).cumsum()

euromomototalcoviddeaths=(pandemic.deaths-pandemic.baseline).cumsum()

data['euromomoexcess']=data.deaths-data.baseline
data['totalcoviddeaths']=pd.Series(totalcoviddeaths)
data['euromomototalcoviddeaths']=pd.Series(euromomototalcoviddeaths)


In [ ]:
#data.merge(covidweekly,on='date',how='outer')

In [ ]:

covidweekly['deathscumulative']=covidweekly.deaths.cumsum()


In [ ]:
coviddeaths=data.deaths-data.noncovidbaseline
data['coviddeaths']=pd.Series(coviddeaths)


fig, ax = plt.subplots(figsize=(15, 8))
#data.iloc[index].plot(x='date',y='coviddeaths',label='Weekly excess above Euromomo baseline',ax=ax,color='blue')

index=data[data.date>=covidweekly.date.min()].index

data.iloc[index-1].plot(x='date',y='euromomoexcess',label='Weekly excess above Euromomo baseline',ax=ax,color='blue')



covidweekly.plot(x='date',y='deaths',label='Weekly official Covid deaths',ax=ax,color='red')


plt.legend()
plt.xlabel('date')# ,fontsize = 20)
plt.ylabel('weekly numbers')# ,fontsize = 20)
plt.title('Weekly deaths caused by lockdown, Covid, other factors')


fig.set_facecolor('w')
plt.tight_layout();
sns.despine;


plt.savefig('plots/weekly_deaths.jpg',dpi=250)

print(data.iloc[index-1].euromomoexcess.sum() - covid.deaths.sum())



In [ ]:
def week2day(df):
    
    start=df.date.min()-timedelta(days=6)
    dailydeaths=pd.Series(np.array([np.repeat(x1/7,7) for x1 in df.deaths.values]).flatten(),name='dailydeaths').to_frame()
    dailybaseline=pd.Series(np.array([np.repeat(x1/7,7) for x1 in df.baseline.values]).flatten(),name='dailybaseline').to_frame()


    date=(pd.Series(pd.date_range(start=start,end=df.date.max()),name='date').to_frame()).reset_index(drop=True)

    df= date.join(dailydeaths)
    df=df.join(dailybaseline)
    
    df['dailyexcess']=df.dailydeaths-df.dailybaseline
    
    return df



df=week2day(data)
df['dailyexcesssmooth']=df.dailyexcess.rolling(7, win_type='gaussian').mean(std=2.5)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))



index=df[df.date>=covid.date.min()].index

df.iloc[index].plot(x='date',y='dailyexcesssmooth',ax=ax,label='Daily excess deaths with respect to Euromomo baseline',color='blue')


covid.plot(x='date',y='deaths',ax=ax,label='Daily official Covid deaths',color='red')

plt.xlabel('date')# ,fontsize = 20)
plt.ylabel('Daily numbers')# ,fontsize = 20)
plt.axhline(0,color='gray',alpha=0.5)

plt.title('Deaths:  Covid, lockdown, other factors?')
#plt.title('EUROMOMO excess deaths during epidemic')

plt.legend()
fig.set_facecolor('w')
plt.tight_layout();
sns.despine;

plt.savefig('plots/Excess_v_Covid_smoothed.png',dpi=250)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))

df['dailyexcesssmooth_cumulative']=df.dailyexcesssmooth.cumsum()
covid['deaths_cumulative']=covid.deaths.cumsum()


index=df[df.date>=covid.date.min()].index

df.iloc[index].plot(x='date',y='dailyexcesssmooth_cumulative',ax=ax,label='Daily excess deaths with respect to Euromomo baseline',color='blue')


covid.plot(x='date',y='deaths_cumulative',ax=ax,label='Daily official Covid deaths',color='red')

plt.xlabel('date')# ,fontsize = 20)
plt.ylabel('Cumulative total')# ,fontsize = 20)
plt.axhline(0,color='gray',alpha=0.5)

plt.title('Deaths:  Covid, lockdown, other factors?')


plt.legend()
fig.set_facecolor('w')
plt.tight_layout();
sns.despine;

plt.savefig('plots/Excess_v_Covid_smoothed_CUMULATIVE.png',dpi=250)

In [ ]:
df2=pd.merge(df,covid,on='date',how='inner')
df2.dropna(inplace=True)
df2.rename(columns={'deaths':'coviddeaths'},inplace=True)
df2['noncoviddeaths']=df2.dailydeaths-df2.coviddeaths
df2['dailydeathssmooth']=df2.dailydeaths.rolling(7, win_type='gaussian').mean(std=2.5)
df2['noncoviddeathssmooth']=df2.dailydeathssmooth-df2.coviddeaths

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))



#index=df[df.date>=covid.date.min()].index

#df.iloc[index].plot(x='date',y='dailyexcesssmooth',ax=ax,label='Daily excess deaths with respect to Euromomo baseline',color='blue')

ax.bar(df2.date,df2.dailydeathssmooth,label='Daily all-cause deaths',color='red',alpha=0.3)
ax.bar(df2.date,df2.coviddeaths,label='Daily official Covid deaths',color='crimson')

#df2.plot.bar(x='date',y='coviddeaths',ax=ax,label='Daily official Covid deaths',color='red')
df2.plot(x='date',y='noncoviddeathssmooth',ax=ax,label='Daily non-Covid deaths',color='purple')
df2.plot(x='date',y='dailybaseline',ax=ax,label='Daily Euromomo baseline',color='gray',alpha=0.9,linestyle=':')

plt.xlabel('date')# ,fontsize = 20)
plt.ylabel('Daily numbers')# ,fontsize = 20)


plt.title('Deaths across Euromomo countries:  Covid and non-Covid \n Data: euromomo.eu & individual national Covid data sets' )
#plt.title('EUROMOMO excess deaths during epidemic')

plt.legend()
fig.set_facecolor('w')
plt.tight_layout();
sns.despine;

plt.savefig('plots/EuromomoCovidnonCovid.png',dpi=250)